[![cloudevel](Imagenes/cloudevel.png)](https://cloudevel.com)

## El archivo ```Dockerfile```.

Docker nos permite crear nuestros propios contenedores mediante el uso de un archivo de configuración llamado ``Dockerfile``. Este archivo es un script, el cual permite crear un contenedor a partir de otro y realizar diversas operaciones con este.

### Comandos de Dockerfile:

* ```FROM``` Es el comando que importa la imagen base a partir de la cual se creará la nueva imagen.
* ```USER``` Define la UISD del usuario que será utilizado.
* ```VOLUME``` Liga a un directorio local con el contenedor.
* ```RUN``` La cual ejecuta diversos comandos al mom,ento de la creación de la imagen.
* ```ADD``` Copia un archivo del sistema de archivos local al sistema de archivos de la imagen.
* ```ENV``` Define variables de entorno.
* ```CMD```Ejecuta comandos al momento de crear un nuevo contenedor a partir de la imagen.
* ```ENTRYPOINT``` Define el comando que se utilizará por defecto al ejecutar un contenedor instanciado de la imagen.
* ```WORKDIR``` 
* ```EXPOSE``` define el reenvío de puertos.



## Construcción de una imagen a partir de un archivo ```Dockerfile```.

Para crear una imagen se utiliza el comando ```docker build``` de la siguiente forma:

```docker build -t <nombre de la imagen> <directorio de la Dockerfile>```

### Ejemplo del archivo Dockerfile.

* El archivo localizado en [src/Dockerfile](src/Dockerfile) contiene lo siguiente:

``` bash
#Descarga la imagen
FROM ubuntu:16.04
 
# Actualiza el sistema
RUN apt-get update
 
# Instala nginx, php-fpm y supervisord
RUN apt-get install -y nginx php7.0-fpm supervisor && \
    rm -rf /var/lib/apt/lists/*
 
#Define las variables de entorno
ENV nginx_vhost /etc/nginx/sites-available/default
ENV php_conf /etc/php/7.0/fpm/php.ini
ENV nginx_conf /etc/nginx/nginx.conf
ENV supervisor_conf /etc/supervisor/supervisord.conf
 
# habilita la configuración del servidor virtual de php-fpm en nginx
COPY default ${nginx_vhost}
RUN sed -i -e 's/;cgi.fix_pathinfo=1/cgi.fix_pathinfo=0/g' ${php_conf} && \
    echo "\ndaemon off;" >> ${nginx_conf}
 
#Copia la configuración del supervisor
COPY supervisord.conf ${supervisor_conf}
 
RUN mkdir -p /run/php && \
    chown -R www-data:www-data /var/www/html && \
    chown -R www-data:www-data /run/php
 
# Configura el volumen de almacenamiento
VOLUME ["/etc/nginx/sites-enabled", "/etc/nginx/certs", "/etc/nginx/conf.d", "/var/log/nginx", "/var/www/html"]
 
# Configura los servicios y los puertos 
COPY start.sh /start.sh
CMD ["./start.sh"]
 
EXPOSE 80 443
```

* El archivo [src/default]([src/default) contiene la configuración de nginx:

```
server {
    listen 80 default_server;
    listen [::]:80 default_server;
 
    root /var/www/html;
    index index.html index.htm index.nginx-debian.html;
 
    server_name _;
 
    location / {
        try_files $uri $uri/ =404;
    }
 
    location ~ \.php$ {
        include snippets/fastcgi-php.conf;
        fastcgi_pass unix:/run/php/php7.0-fpm.sock;
    }
 
    # deny access to .htaccess files, if Apache's document root
    # concurs with nginx's one
    #
    #location ~ /\.ht {
    #    deny all;
    #}
}
```

* El archivo [src/supervisord](src/supervisord) contiene lo siguiente:

```
[unix_http_server]
file=/dev/shm/supervisor.sock   ; (the path to the socket file)
 
[supervisord]
logfile=/var/log/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=false               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)
user=root             ;
 
; the below section must remain in the config file for RPC
; (supervisorctl/web interface) to work, additional interfaces may be
; added by defining them in separate rpcinterface: sections
[rpcinterface:supervisor]
supervisor.rpcinterface_factory = supervisor.rpcinterface:make_main_rpcinterface
 
[supervisorctl]
serverurl=unix:///dev/shm/supervisor.sock ; use a unix:// URL  for a unix socket
 
; The [include] section can just contain the "files" setting.  This
; setting can list multiple files (separated by whitespace or
; newlines).  It can also contain wildcards.  The filenames are
; interpreted as relative to this file.  Included files *cannot*
; include files themselves.
 
[include]
files = /etc/supervisor/conf.d/*.conf
 
 
[program:php-fpm7.0]
command=/usr/sbin/php-fpm7.0 -F
numprocs=1
autostart=true
autorestart=true
 
[program:nginx]
command=/usr/sbin/nginx
numprocs=1
autostart=true
autorestart=true
```

* El archivo [src/start.sh](src/supervisord) tiene permisos de ejecución y contiene lo siguiente:

```
#!/bin/sh
 
/usr/bin/supervisord -n -c /etc/supervisor/supervisord.conf
```

### Creación de la imagen a partir del Dockerfile.

In [1]:
cd src

In [2]:
sudo docker build -t nginx_image .

Sending build context to Docker daemon  13.31kB
Step 1/15 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu

2c1070cd: Pulling fs layer 
74db61f1: Pulling fs layer 
cb72e5c9: Pulling fs layer 
Digest: sha256:e4a134999bea4abb4a27bc437e6118fdddfb172e1b9d683129b74d254af51675
Status: Downloaded newer image for ubuntu:16.04
 ---> 7e87e2b3bf7a
Step 2/15 : RUN apt-get update
 ---> Running in 32bdf31bc404
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:3 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [767 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial/main amd64 Packages [1558 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial/restricted amd64 Packages [14.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial/universe amd64 Pa

Get:42 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libgd3 amd64 2.1.1-4ubuntu0.16.04.10 [126 kB]
Get:43 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libxslt1.1 amd64 1.1.28-2.1ubuntu0.1 [145 kB]
Get:44 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 nginx-common all 1.10.3-0ubuntu0.16.04.3 [26.7 kB]
Get:45 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 nginx-core amd64 1.10.3-0ubuntu0.16.04.3 [429 kB]
Get:46 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 nginx all 1.10.3-0ubuntu0.16.04.3 [3506 B]
Get:47 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 php-common all 1:35ubuntu6.1 [10.8 kB]
Get:48 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 php7.0-common amd64 7.0.32-0ubuntu0.16.04.1 [840 kB]
Get:49 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 php7.0-json amd64 7.0.32-0ubuntu0.16.04.1 [16.9 kB]
Get:50 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 php7.0-opcache amd64 7.0.3

Preparing to unpack .../libfreetype6_2.6.1-0.1ubuntu2.3_amd64.deb ...
Unpacking libfreetype6:amd64 (2.6.1-0.1ubuntu2.3) ...
Selecting previously unselected package libfontconfig1:amd64.
Preparing to unpack .../libfontconfig1_2.11.94-0ubuntu1.1_amd64.deb ...
Unpacking libfontconfig1:amd64 (2.11.94-0ubuntu1.1) ...
Selecting previously unselected package libjpeg8:amd64.
Preparing to unpack .../libjpeg8_8c-2ubuntu8_amd64.deb ...
Unpacking libjpeg8:amd64 (8c-2ubuntu8) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../libtiff5_4.0.6-1ubuntu0.5_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.6-1ubuntu0.5) ...
Selecting previously unselected package libvpx3:amd64.
Preparing to unpack .../libvpx3_1.5.0-2ubuntu1_amd64.deb ...
Unpacking libvpx3:amd64 (1.5.0-2ubuntu1) ...
Selecting previously unselected package libxpm4:amd64.
Preparing to unpack .../libxpm4_1%3a3.5.11-1ubuntu0.16.04.1_amd64.deb ...
Unpacking libxpm4:amd64 (1:3.5.11-1ubuntu0.16.04.1) ...
Selecting prev

debconf: falling back to frontend: Teletype
Setting up nginx-core (1.10.3-0ubuntu0.16.04.3) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up nginx (1.10.3-0ubuntu0.16.04.3) ...
Setting up php-common (1:35ubuntu6.1) ...
Setting up php7.0-common (7.0.32-0ubuntu0.16.04.1) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7.)
debconf: falling bac

debconf: falling back to frontend: Teletype

Creating config file /etc/php/7.0/mods-available/shmop.ini with new version
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Readline.pm line 7.)
debconf: falling back to frontend: Teletype

Creating config file /etc/php/7.0/mods-available/sockets.ini with new version
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debco

invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up python-pkg-resources (20.7.0-1) ...
Setting up python-meld3 (1.0.2-2) ...
Setting up supervisor (3.2.0-2ubuntu0.2) ...
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Processing triggers for sgml-base (1.26+nmu4ubuntu1) ...
Processing triggers for systemd (229-4ubuntu21.15) ...
Removing intermediate container 6c109bf779ca
 ---> 66798d24b088
Step 4/15 : ENV nginx_vhost /etc/nginx/sites-available/default
 ---> Running in 5a4106722fb2
Removing intermediate container 5a4106722fb2
 ---> eed861b42baf
Step 5/15 : ENV php_conf /etc/php/7.0/fpm/php.ini
 ---> Running in 94e72e69e785
Removing intermediate container 94e72e69e785
 ---> 899d496cbb35
Step 6/15 : ENV nginx_conf /etc/nginx/nginx.conf
 ---> Running in 3cb98dd98dbf
Removing intermediate container 3cb98dd98dbf
 ---> 8af2a

In [3]:
sudo docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nginx_image         latest              4a5425ae84f2        1 second ago        246MB
ubuntu              16.04               7e87e2b3bf7a        25 hours ago        117MB
ubuntu              latest              20bb25d32758        25 hours ago        87.5MB
hello-world         latest              fce289e99eb9        3 weeks ago         1.84kB


### Creación de un contenedor a partir de la nueva imagen.

In [15]:
mkdir webroot

mkdir: cannot create directory ‘webroot’: File exists


: 1

In [16]:
sudo docker run -d -v webroot:/var/www/html -p 80:80 --name servidor nginx_image

c12ba24d8eb6dbda12d5fa907c2a5dd210230a768b9c38f0d36e7890a50cd827


* Si se está ejecutando  esta notebook direactamente desde su equipo, el resultado de la ejecución de este contenedor puede ser vista desde http://localhost
* Si se está ejecutando  esta notebook desde la VM provista del sitio de Cloudevel®, el resultado de la ejecución de este contenedor puede ser vista desde http://localhost:8980

In [17]:
sudo docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS                         NAMES
c12ba24d8eb6        nginx_image         "./start.sh"        6 seconds ago       Up 5 seconds        0.0.0.0:80->80/tcp, 443/tcp   servidor


In [7]:
sudo docker stop servidor

servidor


In [8]:
sudo docker rm servidor

servidor


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>